In [1]:
import polaris as po
import numpy as np

# Load the competition from the Hub
competition = po.load_competition("asap-discovery/antiviral-potency-2025")

# Get the train and test data-loaders
train, test = competition.get_train_test_split()

[2025-02-23 23:45:11] INFO     The version of Polaris that was used to create the artifact (0.11.0) ]8;id=323007;file:///home/wim/anaconda3/envs/my-rdkit-env/lib/python3.12/site-packages/polaris/_artifact.py\_artifact.py]8;;\:]8;id=743997;file:///home/wim/anaconda3/envs/my-rdkit-env/lib/python3.12/site-packages/polaris/_artifact.py#92\92]8;;\
                               is different from the currently installed version of Polaris                        
                               (0.11.9).                                                                           

                      WARNING  You're loading data from a remote location. If the dataset is small     ]8;id=667358;file:///home/wim/anaconda3/envs/my-rdkit-env/lib/python3.12/site-packages/polaris/dataset/_base.py\_base.py]8;;\:]8;id=43546;file:///home/wim/anaconda3/envs/my-rdkit-env/lib/python3.12/site-packages/polaris/dataset/_base.py#181\181]8;;\
                               enough, consider caching the dataset first using                                    
                               CompetitionSpecification.cache() for more performant data access.                   

In [3]:
smis = [t[0] for t in train]
y1 = [t[1]['pIC50 (SARS-CoV-2 Mpro)'] for t in train]
y2 = [t[1]['pIC50 (MERS-CoV Mpro)'] for t in train]

In [7]:
test_smis = [t for t in test]

In [18]:
y1 = [4 if np.isnan(y) else y for y in y1]
y2 = [4 if np.isnan(y) else y for y in y2]

In [69]:
#tNN thresholded nearest neighbor
from rdkit import Chem
from rdkit.Chem import rdFingerprintGenerator

MFPGEN = rdFingerprintGenerator.GetMorganGenerator(3,fpSize=4096)

def get_rep(smi):
    m = Chem.MolFromSmiles(smi)
    fp = MFPGEN.GetCountFingerprint(m)
    return fp

X = [get_rep(smi) for smi in smis]

In [86]:
from rdkit import DataStructs
from sklearn.metrics import mean_absolute_error


t = 0.64
probe_fps = [get_rep(smi) for smi in test_smis]
ref_fps = X
ref_y1 = y1
ref_y2 = y2

preds1 = []
preds2 = []
y_pred = {}
for j,fp in enumerate(probe_fps):
    TSs = DataStructs.BulkTanimotoSimilarity(fp,ref_fps)
    idx = [i for i,TS in enumerate(TSs) if TS>t and i!=j]
    if len(idx) == 0:
        idx = [np.argsort(TSs)[-1]]
    pred1 = np.average([ref_y1[v] for v in idx if ref_y1[v]!=4])
    if np.isnan(pred1):
        preds1.append(4)
    else:
        preds1.append(pred1)
    pred2 = np.average([ref_y2[v] for v in idx if ref_y2[v]!=4])
    if np.isnan(pred2):
        preds2.append(4)
    else:
        preds2.append(pred2)
y_pred['pIC50 (SARS-CoV-2 Mpro)'] = preds1
y_pred['pIC50 (MERS-CoV Mpro)'] = preds2


In [88]:
competition.submit_predictions(
    predictions=y_pred,
    prediction_name="potency-thresholded-NN",
    prediction_owner="wim0",
    report_url="https://github.com/dehaenw", 
    # The below metadata is optional, but recommended.
    description="Source code will be made public once challenge is finished",
    user_attributes={"Framework": "RDKit", "Method": "thresholded nearest neighbor"}
)

Output()

[21:56:33]  Success: Submitting competition predictions                                               ]8;id=140681;file:///home/wim/anaconda3/envs/my-rdkit-env/lib/python3.12/site-packages/polaris/utils/context.py\context.py]8;;\:]8;id=316087;file:///home/wim/anaconda3/envs/my-rdkit-env/lib/python3.12/site-packages/polaris/utils/context.py#53\53]8;;\